In [1]:
import pandas as pd

In [71]:
foods = pd.read_csv('data/food.csv')
nutrients = pd.read_csv('data/food_nutrient.csv', usecols=['fdc_id', 'nutrient_id', 'amount'])
portions = pd.read_csv('data/food_portion.csv')

In [72]:
nutrients.head()

,fdc_id,nutrient_id,amount
0,319877,1051,56.30
1,319877,1002,1.28
2,319877,1004,19.00
3,319877,1007,1.98
4,319878,1091,188.00


In [73]:
nutrients_to_save = {
    1003: 'protein',
    1004: 'fat',
    1005: 'carbs',
    1008: 'kcal'
}

## Only selected nutrients
nutrients = nutrients[nutrients['nutrient_id'].isin(nutrients_to_save.keys())]

In [84]:
### one row per food_id with all nutrients as columns
nutrients_pivot = nutrients.pivot(index='fdc_id',
                            columns='nutrient_id',
                            values='amount'
                            ).rename(columns=nutrients_to_save)

In [85]:
nutrients_pivot.head(10)

nutrient_id,protein,fat,carbs,kcal
fdc_id,,,,
319877,NaN,19.0,NaN,NaN
319882,NaN,18.7,NaN,NaN
319892,NaN,16.6,NaN,NaN
319899,NaN,19.1,NaN,NaN
319908,NaN,18.2,NaN,NaN
319928,NaN,18.0,NaN,NaN
319946,NaN,18.2,NaN,NaN
319967,NaN,14.8,NaN,NaN
319981,NaN,11.9,NaN,NaN


In [75]:
unique_portions = portions.sort_values('gram_weight').groupby('fdc_id').first()['gram_weight']

In [76]:
result = foods[['fdc_id', 'description']].merge(nutrients_pivot, on='fdc_id', how='left')
result = result.merge(unique_portions, on='fdc_id', how='left')

In [77]:
result['average_portion'] = result['gram_weight'].fillna(100.0) # Jak nie ma porcji, daj 100g
result = result.fillna(0) # Zera zamiast pustych makro
result['manufacturer'] = "USDA"

In [78]:
print(f"Number of products before uniqueness: {len(result)}")
result = result.drop_duplicates(subset=['description', 'protein', 'fat', 'carbs', 'kcal'])
print(f"Number of products after uniqueness: {len(result)}")

Number of products before uniqueness: 74175


KeyError: Index(['fat', 'kcal', 'carbs', 'protein'], dtype='object')

In [46]:
result.to_json('generic_products_usda.json', orient='records', indent=2)

print("Gotowe! Plik generic_products_usda.json utworzony.")

Gotowe! Plik generic_products_usda.json utworzony.
